In [1]:
import sys
sys.path.append('/content/FreewayGame/leonardo/')
%matplotlib inline
from collections import defaultdict
from typing import List

import numpy as np
import time
import matplotlib.pyplot as plt
import seaborn as sns

import gym

import src.agents as agents
import src.episode as episode
import src.environment as environment
import src.aux_plots as aux_plots

In [2]:
def print_result(i, scores, total_reward, score):
    if i % 10 == 0:
        print(f"Run [{i:4}] - Total reward: {total_reward:7.2f} Mean scores: {sum(scores) / len(scores):.2f} Means Scores[:-10]: {sum(scores[-10:]) / len(scores[-10:]):5.2f} Score: {score:2} ")

In [3]:
RAM_mask = [
      14  # Chicken Y
    , 16  # Chicken Lane Collide
    , 108, 109, 110, 111, 112, 113, 114, 115, 116, 117  # Car X Coords
]

In [4]:
def reduce_state(ob):
    # Doesn't matter where we were hit
    ob[16] = 1 if ob[16] != 255 else 0

    # Reduce chicken y-position
    ob[14] = ob[14] // 3

    for b in range(108, 118):
        # The chicken is in the x-posistion ~49
        if ob[b] < 20 or ob[b] > 80:
            # We don't need to represent cars far from the chicken
            ob[b] = 0
        else:
            # Reduce the cars x-positions sample space
            ob[b] = ob[b] // 3

    return ob
    
def reward_policy(reward, ob, action):
    if reward == 1:
        reward = reward_policy.REWARD_IF_CROSS
    elif ob[16] == 1:  # Collision!
        reward = reward_policy.REWARD_IF_COLISION
    elif action != 1:  # Don't incentivate staying still
        reward = reward_policy.REWARD_IF_STILL

    return reward

In [5]:
env, initial_state = environment.get_env()


In [6]:
GAMMA = 0.99

AVAILABLE_ACTIONS = 2
N0 = 2.5
ALPHA = 0.001

reward_policy.REWARD_IF_CROSS = 1
reward_policy.REWARD_IF_COLISION = -1
reward_policy.REWARD_IF_STILL = -0.0001

# reward_policy.REWARD_IF_CROSS = 500
# reward_policy.REWARD_IF_COLISION = -10
# reward_policy.REWARD_IF_STILL = -1

feat_type = 'all' #mean
agent = agents.SarsaLFAADAM(gamma=GAMMA, state_size=len(RAM_mask), available_actions=AVAILABLE_ACTIONS, N0=N0, discount_factor=1, alpha=ALPHA)
agent.trainScaler(env, RAM_mask, feat_type=feat_type)


In [7]:
scores = []
total_rewards = []

In [ ]:
%%time
n_runs = 500

for i in range(n_runs):
    render = i % 200 == 199

    E = defaultdict(lambda: np.zeros(2)) # TODO available actions

    game_over = False
    state = env.reset()
    state = reduce_state(state)[RAM_mask].data.tobytes()  # Select useful bytes
    action = agent.act(state)
    
    score = 0
    total_reward = 0

    while not game_over:
        if render:
            time.sleep(0.005)
            env.render()

        old_state = state
        old_action = action
        ob, reward, game_over, _ = env.step(action)

        ob = reduce_state(ob)
        reward = reward_policy(reward, ob, action)

        total_reward += reward

        if reward == reward_policy.REWARD_IF_CROSS:
            score += 1

        state = ob[RAM_mask].data.tobytes()

        action = agent.act(state)  # Next action
        
        E[old_state][old_action] += 1

        agent.update_Q(old_s=old_state, new_s=state, old_a=old_action, new_a=action, reward=reward, E=i+1)

    scores.append(score)
    total_rewards.append(total_reward)

    print_result(i, scores, total_reward, score)


Run [   0] - Total reward:  -44.12 Mean scores: 13.00 Means Scores[:-10]: 13.00 Score: 13 
Run [  10] - Total reward:  -51.12 Mean scores: 12.27 Means Scores[:-10]: 12.20 Score: 12 
Run [  20] - Total reward:  -76.10 Mean scores: 12.33 Means Scores[:-10]: 12.40 Score: 12 
Run [  30] - Total reward:  -63.10 Mean scores: 13.03 Means Scores[:-10]: 14.50 Score: 15 
Run [  40] - Total reward:  -55.09 Mean scores: 13.63 Means Scores[:-10]: 15.50 Score: 15 
Run [  50] - Total reward:  -57.09 Mean scores: 14.18 Means Scores[:-10]: 16.40 Score: 16 


In [ ]:
aux_plots.plot_scores(scores)